In [ ]:
"""
These are all the headers of the various tables
"""

misc_headers = ['Team', 'age', 'wins', 'losses', 'pyth wins', 'pyth losses', 'MOV', 'SOS', 'SRS', \
                'ORtg', 'DRtg', 'Pace', 'fta_per_fga_pct', 'fg3a_per_fga_pct', 'ts_pct', 'efg_pct', \
                'tov_pct', 'orb_pct', 'ft_rate', 'opp_efg_pct', 'opp_tov_pct', 'drb_pct', 'opp_ft_rate', \
                'arena_name', 'attendance', 'attendance_per_g']

opponent_headers = ['Team', 'opp Games', 'opp Minutes', 'opp Field Goals', 'opp Field Goal Attempts', \
                    'opp Field Goal Percentage', 'opp 3-Point Field Goals', 'opp 3-Point Field Goal Attempts',\
                    'opp 3-Point Field Goal Percentage', 'opp 2-Point Field Goals', \
                    'opp 2-Point Field Goal Attempts', 'opp 2-Point Field Goal Percentage', 'opp Free Throws', \
                    'opp Free Throw Attempts', 'opp Free Throw Percentage', 'opp Offensive Rebounds', \
                    'opp Defensive Rebounds', 'opp Total Rebounds', 'opp Assists', 'opp Steals', 'opp Blocks',\
                    'opp Turnovers', 'opp Fouls', 'opp Points']


opponent_per100_headers = ['Team', 'opp per100 Games', 'opp per100 Minutes', 'opp per100 Field Goals', \
                    'opp per100 Field Goal Attempts', 'opp per100 Field Goal Percentage', \
                    'opp per100 3-Point Field Goals', 'opp per100 3-Point Field Goal Attempts',\
                    'opp per100 3-Point Field Goal Percentage', 'opp per100 2-Point Field Goals', \
                    'opp per100 2-Point Field Goal Attempts', 'opp per100 2-Point Field Goal Percentage', \
                    'opp per100 Free Throws', 'opp per100 Free Throw Attempts', 'opp per100 Free Throw Percentage',\
                    'opp per100 Offensive Rebounds', 'opp per100 Defensive Rebounds', 'opp per100 Total Rebounds',\
                    'opp per100 Assists', 'opp per100 Steals', 'opp per100 Blocks', 'opp per100 Turnovers', \
                    'opp per100 Fouls', 'opp per100 Points']


team_headers = ['Team', 'Games', 'Minutes', 'Field Goals', 'Field Goal Attempts', 'Field Goal Percentage', \
          '3-Point Field Goals', '3-Point Field Goal Attempts', '3-Point Field Goal Percentage', \
          '2-Point Field Goals', '2-Point Field Goal Attempts', '2-Point Field Goal Percentage', \
          'Free Throws', 'Free Throw Attempts', 'Free Throw Percentage', 'Offensive Rebounds', \
          'Defensive Rebounds', 'Total Rebounds', 'Assists', 'Steals', 'Blocks', 'Turnovers', \
          'Fouls', 'Points']

per100_headers = ['Team', 'per100 Games', 'per100 Minutes', 'per100 Field Goals', 'per100 Field Goal Attempts',\
                  'per100 Field Goal Percentage', 'per100 3-Point Field Goals', 'per100 3-Point Field Goal Attempts',\
                  'per100 3-Point Field Goal Percentage', 'per100 2-Point Field Goals', \
                  'per100 2-Point Field Goal Attempts', 'per100 2-Point Field Goal Percentage', 'per100 Free Throws',\
                  'per100 Free Throw Attempts', 'per100 Free Throw Percentage', 'per100 Offensive Rebounds', \
                  'per100 Defensive Rebounds', 'per100 Total Rebounds', 'per100 Assists', 'per100 Steals', \
                  'per100 Blocks', 'per100 Turnovers', 'per100 Fouls', 'per100 Points']


player_headers = ['name', 'position', 'height', 'weight', 'birthday', 'country', 'exp', 'college']

final_headers = ['Team', 'avg_height', 'avg_weight', 'avg_exp',\
                 'coach', 'players']


coach_headers = ['year', 'age', 'league', 'team', 'games', 'wins', 'losses', 'win_loss_pct', 'wins_over_500', \
                'rank_team', 'g_playoffs', 'wins_playoffs', 'losses_playoffs', 'win_loss_pct_playoffs', \
                'coach_remarks']

In [ ]:
"""
These functions will be used in my scraping
"""

import pandas as pd
import numpy as np
from functools import reduce

def extract_tables(table, headers):
    
    data = []
    
    number_rows = len(table) / len(headers)
    
    for j in range(int(number_rows) - 1):
        single_row = []
        for i in range(len(headers)):
            index = j*len(headers) + i
            single_row.append(table[index].text)
        dict1 = dict(zip(headers, single_row))
        data.append(dict1)
    
    df = pd.DataFrame(data)
    return df


def extract_tables2(table, headers):
    
    data = []
    
    number_rows = len(table) / len(headers)
    
    for j in range(int(number_rows)):
        single_row = []
        for i in range(len(headers)):
            index = j*len(headers) + i
            single_row.append(table[index].text)
        dict1 = dict(zip(headers, single_row))
        data.append(dict1)
    
    df = pd.DataFrame(data)
    return df



def convert_height(list_of_heights): # each item in list has form [feet, inches]
    heights = [[int(i) for i in height] for height in list_of_heights]
    heights = [reduce(lambda x, y: x*12+y, height) for height in heights]
    heights = np.array(heights)
    return heights

def create_dict(list1, headers):
    dict1 = dict(zip(headers, list1))
    
    return dict1

In [ ]:
"""
This code scraps the data I need
"""

import requests
from bs4 import BeautifulSoup
import headers
import re
import time
import pickle


def coach_page(coach_url, coach):
    url2 = 'https://www.basketball-reference.com{}'
    real_url2 = url2.format(coach_url)
    response = requests.get(real_url2)
    
    page = response.text
    page = page.replace('<!--','')
    page = page.replace('-->','')
    soup = BeautifulSoup(page, "lxml")
    
    coach_stats = soup.find(id='all_coach-stats').find_all('tr')
    
    coach_data = []
    
    # Loop through the coach stats table and pull data
    for i in range(2, len(coach_stats)):
        prog = re.compile(r'Coach|Scout|Development')
        result = prog.search(coach_stats[i].text)
        if not result:
            if coach_stats[i].text[:6] == 'Career':
                break
            single_team = []
            year = coach_stats[i].find_all('th')[0].text
            single_team.append(year)
            coach_stats2 = coach_stats[i].find_all('td')
            for j in range(len(headers.coach_headers)-1):
                single_team.append(coach_stats2[j].text)
            coach_dict = dict(zip(headers.coach_headers, single_team))
            coach_data.append(coach_dict)
    coach_df = pd.DataFrame(coach_data)
    
    coach_df[['age','g_playoffs', 'games', 'losses', 'losses_playoffs', 'wins', 'wins_playoffs']] = \
        coach_df[['age','g_playoffs', 'games', 'losses', 'losses_playoffs', 'wins', 'wins_playoffs']].apply(pd.to_numeric)
    
    coach_df['name'] = coach
    
    # Pull the DOB info
    dob_spot = soup.find(id='info').find_all('p')
    for i in range(len(dob_spot)):
        if dob_spot[i].text[:5] == 'Born:':
            dob = dob_spot[i].text[6:]
            coach_df['dob'] = dob
            break
        else:
            coach_df['dob'] = '?'
    
    
    return coach_df



def individual_team_page(team_urls, dict_url_name):
    
    url2 = 'https://www.basketball-reference.com{}'
    list_team_player_info = []

    for team_url in team_urls:
        real_url2 = url2.format(team_url)
        response = requests.get(real_url2)
        
        page = response.text
        page = page.replace('<!--','')
        page = page.replace('-->','')
        soup = BeautifulSoup(page, "lxml")
        
        teamplayers = soup.find(id='all_roster').find_all('td')
        
        player_df = extract_tables(teamplayers, headers.player_headers)
        player_df['exp'] = player_df['exp'].replace('R', '0')
        
        players = player_df['name'].tolist()
        
        # convert heights to total inches
        heights = player_df['height'].str.split('-').tolist()
        heights = convert_height(heights)
        
        # pull metrics we're interested in
        avg_weight = player_df['weight'].astype(int).mean()
        avg_exp = player_df['exp'].astype(int).mean()
        avg_height = np.mean(heights)
        name = dict_url_name[team_url]
        coach = soup.find(text='Coach:').findNext().text
        
        # append coach information to coach table
        coach_url = soup.find(text='Coach:').findNext()['href']
        coach_df = coach_page(coach_url, coach)
        overall_coach_list.append(coach_df)
        
        
        
        # create list for specific team and add to overall list
        player_info_list = [name, avg_height, avg_weight, avg_exp, \
                            coach, players]
        player_info_dict = create_dict(player_info_list, 
                                       headers.final_headers)
        
        list_team_player_info.append(player_info_dict)
    
    df = pd.DataFrame(list_team_player_info)
    return df
        
        
        



def season_page(year, url):
    

    real_url = url.format(year)
    
    response = requests.get(real_url)
    
    
    page = response.text
    page = page.replace('<!--','')
    page = page.replace('-->','')
    soup = BeautifulSoup(page, "lxml")
    
    """
    Pull team stat tables from year page
    """
    team_table = soup.find(id='all_team-stats-per_game').find_all('td')
    opponent_table = soup.find(id='all_opponent-stats-per_game').find_all('td')
    misc_table = soup.find(id='all_misc_stats').find_all('td')
    opponent_per100_table = soup.find(id='all_opponent-stats-per_poss').find_all('td')
    per100_table = soup.find(id='all_team-stats-per_poss').find_all('td')
    
    
    
    """
    Pull team urls and names
    """
    team_urls = []
    team_name_table = soup.find(id='all_misc_stats').find_all('a')
    for a in team_name_table:
        team_urls.append(a['href'])
        
    team_names = []
    team_name_table = soup.find(id='all_misc_stats').find_all('a')
    for a in team_name_table:
        team_names.append(a.text)
        
    dict_url_name = create_dict(team_names, team_urls)
        
        
    """
    Cycle through all teams for that season
    """    
    team_player_info_df = individual_team_page(team_urls, dict_url_name)
    
    
    
    """
    Create dataframes for each table
    """
    team_df = extract_tables(team_table, headers.team_headers)
    opponent_df = extract_tables(opponent_table, headers.opponent_headers)
    misc_df = extract_tables(misc_table, headers.misc_headers)
    opponent_per100_df = extract_tables2(opponent_per100_table, headers.opponent_per100_headers)
    per100_df = extract_tables2(per100_table, headers.per100_headers)
    
    
    """
    Return dataframes
    """
    return team_df, opponent_df, misc_df, opponent_per100_df, per100_df, team_player_info_df
    
    
    

    
    
# create dataframe shells
overall_coach_list = []
overall_team_list = []
overall_opp_list = []
overall_misc_list = []
overall_opp_per100_list = []
overall_per100_list = []
overall_team_player_list = []


years = list(range(2018, 1978, -1))
iters = 0

url = 'https://www.basketball-reference.com/leagues/NBA_{}.html'

for year in years:
    
    # pull current year's info
    team_df1, opp_df1, misc_df1, opp_per100_df1, per100_df1, \
    team_player_df1 = season_page(year, url)
    
    team_df1['year'] = year
    opp_df1['year'] = year
    misc_df1['year'] = year
    opp_per100_df1['year'] = year
    per100_df1['year'] = year
    team_player_df1['year'] = year
    
    overall_team_list.append(team_df1)
    overall_opp_list.append(opp_df1)
    overall_misc_list.append(misc_df1)
    overall_opp_per100_list.append(opp_per100_df1)
    overall_per100_list.append(per100_df1)
    overall_team_player_list.append(team_player_df1)
    
    iters += 1
    if iters % 5 == 0:
        
        overall_team_df = pd.concat(overall_team_list, ignore_index=True)
        overall_opp_df = pd.concat(overall_opp_list, ignore_index=True)
        overall_misc_df = pd.concat(overall_misc_list, ignore_index=True)
        overall_opp_per100_df = pd.concat(overall_opp_per100_list, ignore_index=True)
        overall_per100_df = pd.concat(overall_per100_list, ignore_index=True)
        overall_team_player_df = pd.concat(overall_team_player_list, ignore_index=True)
        overall_coach_df = pd.concat(overall_coach_list, ignore_index=True)
        
        

        export_name1 = 'overall_team_df_{}.pkl'
        export_name1 = export_name1.format(iters)

        export_name2 = 'overall_opp_df_{}.pkl'
        export_name2 = export_name2.format(iters)

        export_name3 = 'overall_misc_df_{}.pkl'
        export_name3 = export_name3.format(iters)

        export_name4 = 'overall_opp_per100_df_{}.pkl'
        export_name4 = export_name4.format(iters)

        export_name5 = 'overall_per100_df_{}.pkl'
        export_name5 = export_name5.format(iters)
        
        export_name6 = 'overall_team_player_df_{}.pkl'
        export_name6 = export_name6.format(iters)        
        
        export_name7 = 'overall_coach_df_{}.pkl'
        export_name7 = export_name7.format(iters)        
    
        with open(export_name1, 'wb') as picklefile:
            pickle.dump(overall_team_df, picklefile)

        with open(export_name2, 'wb') as picklefile:
            pickle.dump(overall_opp_df, picklefile)

        with open(export_name3, 'wb') as picklefile:
            pickle.dump(overall_misc_df, picklefile)

        with open(export_name4, 'wb') as picklefile:
            pickle.dump(overall_opp_per100_df, picklefile)

        with open(export_name5, 'wb') as picklefile:
            pickle.dump(overall_per100_df, picklefile)

        with open(export_name6, 'wb') as picklefile:
            pickle.dump(overall_team_player_df, picklefile)

        with open(export_name7, 'wb') as picklefile:
            pickle.dump(overall_coach_df, picklefile)
            
        overall_coach_list = []
        overall_team_list = []
        overall_opp_list = []
        overall_misc_list = []
        overall_opp_per100_list = []
        overall_per100_list = []
        overall_team_player_list = []
        
        del overall_team_df
        del overall_opp_df
        del overall_misc_df
        del overall_opp_per100_df
        del overall_per100_df
        del overall_team_player_df
        del overall_coach_df
        
        time.sleep(10)